In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import utils
import tqdm
import dataloaders
import torchvision
from trainer import Trainer

### Model Definition

In [2]:
torch.random.manual_seed(0)

class FullyConnectedModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # We are using 28x28 greyscale images.
        num_input_nodes = 28*28
        # Number of classes in the MNIST dataset
        num_classes = 10

        # Define our model
        self.classifier = torch.nn.Sequential(
            # torch.nn.Linear(num_input_nodes, 64), 
            # torch.nn.ReLU(), 
            # torch.nn.Linear(64, num_classes),
            torch.nn.Linear(num_input_nodes, num_classes),
        )
    
    def forward(self, x):
        # Runs a forward pass on the images
        x = x.view(-1, 28*28)
        out = self.classifier(x)
        return out
    
class FullyConnectedModelWithHiddenLayer(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # We are using 28x28 greyscale images.
        num_input_nodes = 28*28
        # Number of hidden nodes
        num_hidden_nodes = 64
        # Number of classes in the MNIST dataset
        num_classes = 10

        # Define our model
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(num_input_nodes, num_hidden_nodes), 
            torch.nn.ReLU(), 
            torch.nn.Linear(num_hidden_nodes, num_classes),
            # torch.nn.Linear(num_input_nodes, num_classes),
        )
    
    def forward(self, x):
        # Runs a forward pass on the images
        x = x.view(-1, 28*28)
        out = self.classifier(x)
        return out

### Hyperparameters & Loss function

In [3]:
# Hyperparameters
batch_size = 64
learning_rate_good = 0.0192
learning_rate_bad  = 1.0
num_epochs = 5


# Use CrossEntropyLoss for multi-class classification
loss_function = torch.nn.CrossEntropyLoss()

# Model definition
original_model = FullyConnectedModel()
better_model   = FullyConnectedModelWithHiddenLayer()

# Define optimizer (Stochastic Gradient Descent)
original_optimizer_good = torch.optim.SGD(original_model.parameters(),
                            lr=learning_rate_good)
original_optimizer_bad = torch.optim.SGD(original_model.parameters(),
                            lr=learning_rate_bad)
better_optimizer = torch.optim.SGD(better_model.parameters(),
                            lr=learning_rate_good)

image_transform_original = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor()
])

image_transform = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor(), 
  torchvision.transforms.Normalize([0.5], [0.25])
])

dataloader_train, dataloader_val = dataloaders.load_dataset(batch_size, image_transform=image_transform)
dataloader_train_original, dataloader_val_original = dataloaders.load_dataset(batch_size, image_transform=image_transform_original)


### Train model

In [4]:
original_trainer_good = Trainer(
  model=original_model,
  dataloader_train=dataloader_train_original,
  dataloader_val=dataloader_val_original,
  batch_size=batch_size,
  loss_function=loss_function,
  optimizer=original_optimizer_good
)
original_trainer_good_normalized = Trainer(
  model=original_model,
  dataloader_train=dataloader_train,
  dataloader_val=dataloader_val,
  batch_size=batch_size,
  loss_function=loss_function,
  optimizer=original_optimizer_good
)
original_trainer_bad = Trainer(
  model=original_model,
  dataloader_train=dataloader_train,
  dataloader_val=dataloader_val,
  batch_size=batch_size,
  loss_function=loss_function,
  optimizer=original_optimizer_bad
)
better_trainer = Trainer(
  model=better_model,
  dataloader_train=dataloader_train,
  dataloader_val=dataloader_val,
  batch_size=batch_size,
  loss_function=loss_function,
  optimizer=better_optimizer
)

original_train_loss_dict_good, original_val_loss_dict_good = original_trainer_good.train(num_epochs)
original_train_loss_dict_good_normalized, original_val_loss_dict_good_normalized = original_trainer_good_normalized.train(num_epochs)
original_train_loss_dict_bad,  original_val_loss_dict_bad  =  original_trainer_bad.train(num_epochs)
better_train_loss_dict,        better_val_loss_dict        =        better_trainer.train(num_epochs)


Training epoch 4: 100%|██████████| 938/938 [00:09<00:00, 99.54it/s]


In [ ]:
weight = next(model.classifier.children()).weight.data
print(weight.shape)

for w in range(weight.shape[0]):
    im = np.zeros((28, 28))
    
    minimum = weight[w, :].min()
    maximum = weight[w, :].max()
    
    for y in range(28):
        for x in range(28):
            im[y, x] = float((weight[w, y * 28 + x] - minimum) / (maximum - minimum))
    
    # plt.imsave("weights/hidden_class_" + str(w) + "_weight_image.jpg", im, cmap="gray")
    print(im)

In [ ]:
# Plot original result
utils.plot_loss(original_train_loss_dict_good, label="Train Loss")
utils.plot_loss(original_val_loss_dict_good, label="Test Loss")
plt.ylim([0, 1])
plt.legend()
plt.xlabel("Number of Images Seen")
plt.ylabel("Cross Entropy Loss")
plt.savefig("original_training_loss.png")

plt.show()
torch.save(original_model.state_dict(), "original_saved_model.torch")
final_loss, final_acc = utils.compute_loss_and_accuracy(
    dataloader_val_original, original_model, loss_function)
print(f"Final Test Cross Entropy Loss: {final_loss}. Final Test accuracy: {final_acc}")


In [ ]:
# Plot original vs normalized results
utils.plot_loss(original_train_loss_dict_good, label="Train Loss")
utils.plot_loss(original_val_loss_dict_good, label="Test Loss")
utils.plot_loss(original_train_loss_dict_good_normalized, label="Train Loss, Normalized")
utils.plot_loss(original_val_loss_dict_good_normalized, label="Test Loss, Normalized")
plt.ylim([0, 1])
plt.legend()
plt.xlabel("Number of Images Seen")
plt.ylabel("Cross Entropy Loss")
plt.savefig("original_vs_normalized_training_loss.png")

plt.show()
torch.save(original_model.state_dict(), "original_saved_model.torch")
final_loss, final_acc = utils.compute_loss_and_accuracy(
    dataloader_val, original_model, loss_function)
print(f"Normalized Final Test Cross Entropy Loss: {final_loss}. Final Test accuracy: {final_acc}")


In [ ]:
# Plot bad learning rate
utils.plot_loss(original_train_loss_dict_bad, label="Train Loss")
utils.plot_loss(original_val_loss_dict_bad, label="Test Loss")
plt.ylim([0, 100])
plt.legend()
plt.xlabel("Number of Images Seen")
plt.ylabel("Cross Entropy Loss")
plt.savefig("bad_learning_rate_training_loss.png")

plt.show()
torch.save(original_model.state_dict(), "original_saved_model.torch")
final_loss, final_acc = utils.compute_loss_and_accuracy(
    dataloader_val, original_model, loss_function)
print(f"Normalized Final Test Cross Entropy Loss: {final_loss}. Final Test accuracy: {final_acc}")

In [ ]:
# Plot normalized vs hidden layer model results
utils.plot_loss(original_train_loss_dict_good_normalized, label="Train Loss")
utils.plot_loss(original_val_loss_dict_good_normalized, label="Test Loss")
utils.plot_loss(better_train_loss_dict, label="Train Loss, Hidden Layer")
utils.plot_loss(better_val_loss_dict, label="Test Loss, Hidden Layer")
plt.ylim([0, 1])
plt.legend()
plt.xlabel("Number of Images Seen")
plt.ylabel("Cross Entropy Loss")
plt.savefig("normalized_vs_hidden_layer_training_loss.png")

plt.show()
torch.save(better_model.state_dict(), "better_saved_model.torch")
final_loss, final_acc = utils.compute_loss_and_accuracy(
    dataloader_val, better_model, loss_function)
print(f"Normalized Final Test Cross Entropy Loss: {final_loss}. Final Test accuracy: {final_acc}")